In [19]:
# Load packages and directories


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import os
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.subplots as sp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import plotly.express as px
import seaborn as sns
import importlib

import preparing_for_bill as pfb

import re
import pickle 
from concurrent.futures import ProcessPoolExecutor
base_directory = "C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning"
data_directory = os.path.join(base_directory, 'edp_data')
cleaned_data_directory = os.path.join(base_directory, 'resampled_30min_data')

# Directory containing the household profile CSV files
survey_data = pd.read_csv(os.path.join(base_directory, 'survey/edp_survey_aircon.csv'))  
output_directory = os.path.join(base_directory, 'figures')  # Change this to your output directory
resampled_data_directory = os.path.join(base_directory, 'resampled_30min_data_real')

filtered_data_directory = os.path.join(base_directory, 'archive/filtered_data_real')

# Directory to save profiles
site_profiles_directory = os.path.join(base_directory, 'full_profiles_real')

resampled_data_directory = os.path.join(base_directory,'resampled_30min_data_real')

In [7]:
with open("testing_site_ids.pkl", "rb") as file:
    testing_site_ids = pickle.load(file)

In [13]:
print(testing_site_ids)

Index(['S0576', 'W0236', 'S0177', 'W0087', 'W0058', 'W0334', 'W0031', 'W0142',
       'S0209', 'W0328', 'S0463', 'W0241', 'S0299', 'W0321', 'W0138', 'S0229',
       'W0061', 'W0329', 'S0249', 'W0326', 'S0179', 'W0037', 'W0158', 'W0275',
       'S0250', 'W0017', 'W0227', 'W0014', 'S0227', 'W0137', 'S0568', 'S0392',
       'W0118', 'W0082', 'W0195', 'W0181', 'S0216', 'W0293', 'W0213', 'W0172',
       'W0166', 'W0081', 'S0509', 'W0266', 'W0342', 'W0011', 'W0073'],
      dtype='object')


In [18]:
to_process_ids = ['W0334', 'W0335', 'W0336', 'W0341', 'W0342', 'W0345', 'W0346', 'W0348', 'W0352']

In [24]:
importlib.reload(pfb)

def process_site(site_id, data_directory, site_filtered_data_directory, site_profiles_directory):
    """
    Process all files for a given site in parallel, filter and aggregate the data, 
    and save the final result to a CSV file.
    """
    # Ensure the site-specific filtered data directory exists
    os.makedirs(site_filtered_data_directory, exist_ok=True)

    # Initialize an empty list to store the aggregated dataframes
    dfs = []

    # Process each file in parallel
    with ProcessPoolExecutor() as executor:
        filepaths = [os.path.join(data_directory, filename) for filename in os.listdir(data_directory) if filename.endswith('.csv')]
        futures = [executor.submit(pfb.process_file, filepath, site_id) for filepath in filepaths]

        # Collect results as they complete
        for future in futures:
            result = future.result()
            if result is not None:
                dfs.append(result)

    # If no data was collected, skip to next site
    if not dfs:
        print(f"No matching data for site {site_id}")
        return

    # Concatenate all dataframes and sort by datetime
    stacked_df = pd.concat(dfs, ignore_index=True)
    stacked_df['datetime'] = pd.to_datetime(stacked_df['datetime'])
    stacked_df.sort_values(by='datetime', inplace=True)

    # Rename columns to the desired format
    stacked_df.rename(columns={'edp_site_id': 'CUSTOMER_ID', 'datetime': 'TS', 'pv_site_net': 'PV', 'ac_load_net': 'kWh', 'load_air_conditioner': 'Air_Conditioner_Load'}, inplace=True)

    # Save the final dataframe to a CSV file
    output_filename = os.path.join(site_profiles_directory, f'{site_id}_profile.csv')
    stacked_df.to_csv(output_filename, index=False)
    print(f"Processed data for site {site_id} saved to {output_filename}")



for site_id in to_process_ids:
    site_filtered_data_directory = os.path.join(filtered_data_directory, site_id)
    process_site(site_id, data_directory, site_filtered_data_directory, site_profiles_directory)

MemoryError: Unable to allocate 256. KiB for an array with shape (32768,) and data type int64

In [16]:

# Iterate over each site ID
for site_id in testing_site_ids:
    # Read the CSV file for the current site ID
    df = pd.read_csv(os.path.join('full_profiles_real', f'{site_id}_profile.csv'))

    # Convert 'TS' column to datetime and set it as index
    df['TS'] = pd.to_datetime(df['TS'])
    df.set_index('TS', inplace=True)

    #limit the dates
    df = df.loc['2018-01-01':'2023-12-31']

    #take the average at each timestamp (site S0013 had double the measurements of a the 12th each month in 2019)
    df = df.groupby(df.index).mean()

    # Resample the data to 30-minute intervals and take the sum of each interval, then *2 to convert from W --> Wh
    resampled_df = df.resample('30T').sum(numeric_only=True)*2
    
    # Convert units from Wh to kWh
    resampled_df['PV'] /= 1000
    resampled_df['kWh'] /= 1000

    # Reset index to make 'TS' a column again
    resampled_df.reset_index(inplace=True)

    # Save the resampled DataFrame to a new CSV file
    output_filename = os.path.join(resampled_data_directory, f'{site_id}_real_profile.csv')
    resampled_df.to_csv(output_filename, index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'full_profiles_real\\W0334_profile.csv'